In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import os
xr.set_options(keep_attrs=True)

In [2]:
mydir = '/glade/work/lettier/CDRv4/day/'
myvariables = ['cdr_seaice_conc','nsidc_bt_seaice_conc','nsidc_nt_seaice_conc']

In [3]:
nh_area = np.fromfile('/glade/work/lettier/GRIDS/psn25area_v3.dat',dtype='<i4').reshape([448,304])/1000
sh_area = xr.open_dataset('/glade/work/lettier/GRIDS/0_pss25area_netcdf_bt_.nc').areacello.values

In [4]:
sample_ds_nh = xr.open_dataset(mydir+'nhdl/1980/seaice_conc_daily_nh_19801231_n07_v04r00.nc').isel(tdim=0)
sample_ds_sh = xr.open_dataset(mydir+'shdl/1980/seaice_conc_daily_sh_19801231_n07_v04r00.nc').isel(tdim=0)

In [5]:
nhgrid = xr.DataArray(nh_area, dims=['y','x'], coords = {'ygrid':sample_ds_nh.ygrid, 'xgrid':sample_ds_nh.xgrid} ).to_dataset(name='areacello')
nhgrid.areacello.attrs['units'] = 'km^2'
nhgrid.areacello.attrs['long_name'] = 'grid_cell_area'
nhgrid.attrs['description'] = 'grid cell area for NSIDC polar stereographic grid 25km, Northern Hemisphere'

shgrid = xr.DataArray(sh_area,dims=['y','x'],coords= {'ygrid':sample_ds_sh.ygrid, 'xgrid':sample_ds_sh.xgrid}).to_dataset(name='areacello')
shgrid.areacello.attrs['units'] = 'km^2'
shgrid.areacello.attrs['long_name'] = 'grid_cell_area'
shgrid.attrs['description'] = 'grid cell area for NSIDC polar stereographic grid 25km, Southern Hemisphere'

In [6]:
def compute_sia_si_nh (myfile):

    ds = xr.open_dataset(datadir+myfile)
    ds = ds[myvariables]
    
    flagdict = {}
    for var in myvariables:
        flagdict[var] = False
        if np.count_nonzero(np.isnan(ds[var]))>0.:
            flagdict[var] = True
        
    ds = ds.where(ds<2.5) # keep only valid data
    ints = ds.where(ds>.15) # for ice extent
    ints = ints/ints
    sie = (ints*nhgrid.areacello).sum(dim=('y','x'))/1e6
    sia = (ds*nhgrid.areacello).sum(dim=('y','x'))/1e6
    
    
    
    for var in myvariables: # if any variables has NaNs, set SIE and SIA to NaN
        if flagdict[var]:
            sia[var].values = [np.nan]
            sie[var].values = [np.nan]

    sia = sia.rename({'cdr_seaice_conc':'sia_nh_cdr','nsidc_bt_seaice_conc':'sia_nh_bt','nsidc_nt_seaice_conc':'sia_nh_nt'})
    sie = sie.rename({'cdr_seaice_conc':'sie_nh_cdr','nsidc_bt_seaice_conc':'sie_nh_bt','nsidc_nt_seaice_conc':'sie_nh_nt'})

    sids = xr.merge([sia, sie])
    for var in sids:
        sids[var].attrs['units'] = 'million km^2'
    sids.attrs = ds.attrs
    sids.attrs['processed'] = 'SIA and SIE computed by Lettie Roach, 2021'
    sids = sids.rename({'tdim':'time'})

    
    return sids


def compute_sia_si_sh (myfile):

    ds = xr.open_dataset(datadir+myfile)
    ds = ds[myvariables]
    
    flagdict = {}
    for var in myvariables:
        flagdict[var] = False
        if np.count_nonzero(np.isnan(ds[var]))>0.:
            flagdict[var] = True
        

    ds = ds.where(ds<2.5) # keep only valid data
    ints = ds.where(ds>.15) # for ice extent
    ints = ints/ints
    sie = (ints*shgrid.areacello).sum(dim=('y','x'))/1e6
    sia = (ds*shgrid.areacello).sum(dim=('y','x'))/1e6
    
    for var in myvariables: # if any variables has NaNs, set SIE and SIA to NaN
        if flagdict[var]:
            sia[var].values = [np.nan]
            sie[var].values = [np.nan]

    sia = sia.rename({'cdr_seaice_conc':'sia_sh_cdr','nsidc_bt_seaice_conc':'sia_sh_bt','nsidc_nt_seaice_conc':'sia_sh_nt'})
    sie = sie.rename({'cdr_seaice_conc':'sie_sh_cdr','nsidc_bt_seaice_conc':'sie_sh_bt','nsidc_nt_seaice_conc':'sie_sh_nt'})

    sids = xr.merge([sia, sie])
    for var in sids:
        sids[var].attrs['units'] = 'million km^2'
    sids.attrs = ds.attrs
    sids.attrs['processed'] = 'SIA and SIE computed by Lettie Roach, 2021'
    sids = sids.rename({'tdim':'time'})

    
    return sids


In [7]:
for year in np.arange(1979,2022,1):
    print(year)
    
    datadir = mydir+'nhdl/'+str(year)+'/'
    myfiles = sorted(os.listdir(datadir))
    
    list_nh = []
    for myfile in myfiles:
        list_nh.append(compute_sia_si_nh (myfile))
    sids_nh = xr.concat(list_nh,dim='time')
        
    list_sh = []
    datadir = mydir+'shdl/'+str(year)+'/'
    myfiles = [f for f in sorted(os.listdir(datadir)) if '.nc' in f]
    for myfile in myfiles:
        list_sh.append(compute_sia_si_sh (myfile))
    sids_sh = xr.concat(list_sh,dim='time')
        
    outds = xr.merge([sids_nh, sids_sh])
   
    outds.to_netcdf(mydir+'/processed/sia_cdr_daily_nhsh_'+str(year)+'_f17_v04r00.nc')

1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
